# More data wrangling

__iClicker:__ Please comment anything about the course.

__iClicker__: Extra office hours beginning next week!

- A. __Monday 7 pm__
- B. Monday 8 pm
- C. Wednesday 7 pm
- D. Wednesday 8 pm



A lot of things covered in the Week 5 lectures:


- Data wrangling with Pandas
  - Row and column manipulation 
    - indexing with `loc` and `iloc`
    - `query()`
    - pivoting - changing rows and columns
    - `drop()`
    - `str` attributes
    - `assign()`
  - Aggregations
    - `groupby()`
    - `agg()` or `aggregate()`
    - `apply()`
  - Joins
    - The `merge()` method

- Visualization
  - Matplotlib
  - Seaborn
  - Plotly
  
- Databases
  - `sqlite3`
  - `DuckDB`

- Web scraping
  - We used `BeautifulSoup`.
  - CSS selectors
  - Some other tools include: `scrapy`, more scalable and has more features. Requires knowledge in OOP. 
  - There are fair amount of webpages involving JavaScript. For example: The pagination through the "next" button from the page
  https://www.imdb.com/search/title/?title_type=feature&release_date=2023-01-01,2023-12-31&count=100
  - `selenium` is the package to use in that case.
  
  
Homework 3 and 4 intensively feature data wrangling, visualization, and some databases. 

Homework 5 will be a web scraping homework featuring autograder. 



Let's discuss a bit more about data wrangling pattern. 

## Transformation

In the last lecture, we had an example selecting first ICU stay of each subject:



```R
icustays_subset <- icustays_tble |>
  # first ICU stay of each unique `subject_id`
  group_by(subject_id) |>
  slice_min(intime) |>
  ungroup() |>
  show_query() |>
  print(width = Inf)
```

In [1]:
sql_cmd = """
SELECT
  subject_id,
  hadm_id,
  stay_id,
  first_careunit,
  last_careunit,
  intime,
  outtime,
  los
FROM (
  SELECT
    *,
    RANK() OVER (PARTITION BY subject_id ORDER BY intime) AS visit_num
  FROM icustays
) 
WHERE (visit_num == 1)
"""

In [26]:
%%time
import pandas as pd
import sqlite3
with sqlite3.connect("mimiciv.sqlite") as conn:
    df = pd.read_sql_query(sql_cmd, conn)
df

CPU times: user 167 ms, sys: 32.8 ms, total: 200 ms
Wall time: 208 ms


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
4,10001884,26184834,37510196,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817
...,...,...,...,...,...,...,...,...
50915,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
50916,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
50917,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
50918,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


We had a new pattern of SQL command, which could not be directly converted to `pandas` commands from what we have learned so far. 

The first `SELECT` query run is:
```sql
SELECT
*,
RANK() OVER (PARTITION BY subject_id ORDER BY intime) AS visit_num
FROM icustays
```
It adds a new column named `visit_num` corresponding to the order of visit within `subject_id`. The `OVER` keyword indicates the use of [window function](https://database.guide/understanding-window-functions-in-sql/). What it does may seem similar to general aggregations, but the catch here is that the number of rows is retained.

- `PARTITION BY` is similar to `GROUP BY`
- `ORDER BY` gives order within the group
- `RANK()` function computes the rank within the partition based on the ordering defined. 


Actually, we can achieve the same thing using the function `transform()`. 

Let's read the `icustays.csv.gz` file as a data frame.

In [3]:
icustays = pd.read_csv("~/mimic/icu/icustays.csv.gz")

In [4]:
icustays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
73177,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
73178,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
73179,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


The `transform()` applies the function to each column of `DataFrame` or `Series`. The returned value has the same size as the original data.

In [5]:
%%time
icustays["visitnum"] = icustays.groupby("subject_id")["intime"].transform("rank")

CPU times: user 77.9 ms, sys: 15.3 ms, total: 93.2 ms
Wall time: 112 ms


In [14]:
icustays["visitnum"] = icustays.groupby("subject_id")["intime"].transform("rank")

In [15]:
icustays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,visitnum
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,1.0
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,1.0
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,1.0
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,2.0
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,1.0
...,...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,1.0
73177,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,1.0
73178,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,1.0
73179,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,1.0


__Exercise__: What pandas command would result in the same `50920 × 8` `DataFrame` as in the previous example?





In [47]:
%%timeit
icustays.query("visitnum == 1")#.drop(columns="visitnum")

2.29 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%%timeit
icustays[icustays["visitnum"] == 1]#.drop(columns="visitnum")

1.94 ms ± 62.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


What you might have noticed is that there are more than one ways to select the minimum value within each group. The approach we just have used is flexible in that we can select `n` samples from each group. 

In [11]:
icustays.groupby("subject_id")["intime"].transform(
    lambda x : x.rank(method="average"))

0        1.0
1        1.0
2        1.0
3        2.0
4        1.0
        ... 
73176    1.0
73177    1.0
73178    1.0
73179    1.0
73180    1.0
Name: intime, Length: 73181, dtype: float64

In fact, `rank` within group is used a lot, so there is a dedicated `rank()` method for it.
It lets you control the behavior on ties (`average` is the default, `min`, `max`, `first`, `dense` available). 

In what other scenarios is `transform()` useful?

cumsum, diff, etc. 

In [17]:
icustays.groupby("subject_id")["intime"].transform(
    lambda x : x.rank(method="max"))

0        1.0
1        1.0
2        1.0
3        2.0
4        1.0
        ... 
73176    1.0
73177    1.0
73178    1.0
73179    1.0
73180    1.0
Name: intime, Length: 73181, dtype: float64

In [18]:
icustays.groupby("subject_id")["intime"].rank(method="max")

0        1.0
1        1.0
2        1.0
3        2.0
4        1.0
        ... 
73176    1.0
73177    1.0
73178    1.0
73179    1.0
73180    1.0
Name: intime, Length: 73181, dtype: float64

In [16]:
icustays.groupby("subject_id")["los"].transform(
    lambda x : (x - x.mean())/x.std())


0             NaN
1             NaN
2        0.707107
3       -0.707107
4             NaN
           ...   
73176         NaN
73177         NaN
73178         NaN
73179         NaN
73180         NaN
Name: los, Length: 73181, dtype: float64

__Problem.__ Select rows of the data frame corresponding to the last ICU visit of each subject. 

In [19]:
icustays["visitnum"] = icustays.groupby("subject_id")["intime"].transform("rank")
icustays["visitcount"] = icustays.groupby("subject_id")["intime"].transform("count")


In [21]:
icustays.query("visitnum == visitcount")

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,visitnum,visitcount
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,1.0,1
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,1.0,1
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,2.0,2
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,1.0,1
5,10001884,26184834,37510196,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,1.0,1
...,...,...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,1.0,1
73177,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,1.0,1
73178,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,1.0,1
73179,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,1.0,1


When to use `transform()`, `agg()`, and `apply()`?

- `transform()` applies a function to each column, and its output must have (or be broadcasted to) the same size as the input
- `agg()` applies a function to each column, and its output must be one number. 
- `apply()` is the most flexible. The function applied may involve multiple columns. However, if something can be done using `agg()` and `transform()`, they will be faster. 

## Filtrations

Here is another approach to get the first hospital visit of each subject:

In [23]:
%%time
(icustays.sort_values(["intime"]).groupby("subject_id")
 .nth(0).sort_values(["subject_id"])
)

CPU times: user 68.8 ms, sys: 4.74 ms, total: 73.5 ms
Wall time: 76 ms


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,visitnum,visitcount
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,1.0,1
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,1.0,1
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,1.0,2
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,1.0,1
5,10001884,26184834,37510196,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,1.0,1
...,...,...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,1.0,1
73177,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,1.0,1
73178,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,1.0,1
73179,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,1.0,1


For the second visits:

In [24]:

(icustays.sort_values(["intime"]).groupby("subject_id")
 .nth(1).sort_values(["subject_id"]))

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,visitnum,visitcount
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,2.0,2
9,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,2.0,3
14,10002428,28662225,38875437,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-19 18:11:19,2156-04-26 18:58:41,7.032894,2.0,4
20,10002930,25922998,35629889,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2198-04-17 21:24:00,2198-04-18 13:41:43,0.678970,2.0,2
24,10003400,23559586,34577403,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-08-10 19:54:51,2137-08-13 17:54:54,2.916701,2.0,3
...,...,...,...,...,...,...,...,...,...,...
73155,19997473,27787494,32134105,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2173-09-17 15:07:55,2173-09-22 06:39:09,4.646690,2.0,2
73165,19998330,24492004,32641669,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2178-10-01 08:51:00,2178-10-03 23:25:08,2.607037,2.0,6
73166,19998591,24349193,31144045,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2185-07-16 18:48:18,2185-07-26 18:27:01,9.985220,2.0,2
73170,19998878,26489544,34403689,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2132-09-30 21:55:00,2132-10-01 15:43:11,0.741794,2.0,2


## Another package worth mentioning: `polars`

Polars is another package worth mentioning. Some of the `pandas` core developers are working on it to create a more performant python-based `DataFrame`. It automatically use multiple cores on your computer, just like `DuckDB` does. It takes advantage of `arrow` memory representation, resulting in efficient and cross-platform data layout. The implementation is backed by the programming language `Rust`. 

Perhaps `polars` and `DuckDB` are two of the hottest topic for single-machine data analytics today. 

- `DuckDB` is heavily based on SQL queries. 
- `polars` is more of a Python programming tool.

DuckDB people maintain benchmark of the performance of queries within a single machine:
https://duckdb.org/2023/11/03/db-benchmark-update.html


Syntax of `polars` highly resembles that of `pandas`, with some minor changes in selection and how to build user-defined queries. It is possible to optimize query and perform lazy evaluation.



From [their documentation](https://docs.pola.rs/):
Key features include: 
- __Fast__: Written from scratch in Rust, designed close to the machine and without external dependencies.
- __I/O__: First class support for all common data storage layers: local, cloud storage & databases.
- __Intuitive API__: Write your queries the way they were intended. Polars, internally, will determine the most efficient way to execute using its query optimizer.
- __Out of Core__: The streaming API allows you to process your results without requiring all your data to be in memory at the same time
- __Parallel__: Utilises the power of your machine by dividing the workload among the available CPU cores without any additional configuration.
- __Vectorized Query Engine__: Using Apache Arrow, a columnar data format, to process your queries in a vectorized manner and SIMD to optimize CPU usage.


Philosophy:
The goal of Polars is to provide a lightning fast DataFrame library that:

- Utilizes all available cores on your machine.
- Optimizes queries to reduce unneeded work/memory allocations.
- Handles datasets much larger than your available RAM.
- A consistent and predictable API.
- Adheres to a strict schema (data-types should be known before running the query).
- Polars is written in Rust which gives it C/C++ performance and allows it to fully control performance critical parts in a query engine.

In [27]:
import polars as pl
from datetime import datetime

df = pl.DataFrame(
    {
        "integer": [1, 2, 3],
        "date": [
            datetime(2025, 1, 1),
            datetime(2025, 1, 2),
            datetime(2025, 1, 3),
        ],
        "float": [4.0, 5.0, 6.0],
        "string": ["a", "b", "c"],
    }
)

print(df)

shape: (3, 4)
┌─────────┬─────────────────────┬───────┬────────┐
│ integer ┆ date                ┆ float ┆ string │
│ ---     ┆ ---                 ┆ ---   ┆ ---    │
│ i64     ┆ datetime[μs]        ┆ f64   ┆ str    │
╞═════════╪═════════════════════╪═══════╪════════╡
│ 1       ┆ 2025-01-01 00:00:00 ┆ 4.0   ┆ a      │
│ 2       ┆ 2025-01-02 00:00:00 ┆ 5.0   ┆ b      │
│ 3       ┆ 2025-01-03 00:00:00 ┆ 6.0   ┆ c      │
└─────────┴─────────────────────┴───────┴────────┘


I encourage you to try this package out for Homework 4 -- this is not enforced, though.

Some example:

In [28]:
icustays_pl = pl.read_csv("~/mimic/icu/icustays.csv.gz")

Lazy evaluation is not available for `csv.gz` yet. 

In [29]:
icustays_pl.head(10)

subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
i64,i64,i64,str,str,str,str,f64
10000032,29079034,39553978,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2180-07-23 14:00:00""","""2180-07-23 23:50:47""",0.410266
10000980,26913865,39765666,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2189-06-27 08:42:00""","""2189-06-27 20:38:27""",0.497535
10001217,24597018,37067082,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2157-11-20 19:18:02""","""2157-11-21 22:08:00""",1.118032
10001217,27703517,34592300,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2157-12-19 15:42:24""","""2157-12-20 14:27:41""",0.948113
10001725,25563031,31205490,"""Medical/Surgical Intensive Car…","""Medical/Surgical Intensive Car…","""2110-04-11 15:52:22""","""2110-04-12 23:59:56""",1.338588
10001884,26184834,37510196,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2131-01-11 04:20:05""","""2131-01-20 08:27:30""",9.171817
10002013,23581541,39060235,"""Cardiac Vascular Intensive Car…","""Cardiac Vascular Intensive Car…","""2160-05-18 10:00:53""","""2160-05-19 17:33:33""",1.314352
10002155,20345487,32358465,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2131-03-09 21:33:00""","""2131-03-10 18:09:21""",0.858576
10002155,23822395,33685454,"""Coronary Care Unit (CCU)""","""Coronary Care Unit (CCU)""","""2129-08-04 12:45:00""","""2129-08-10 17:02:38""",6.178912


In [38]:
icustays_pl.group_by("subject_id").len() # Note that the result changes each run.

subject_id,len
i64,u32
10000032,1
10000980,1
10001217,2
10001725,1
10001884,1
…,…
19999442,1
19999625,1
19999828,1


In [39]:
pl.__version__

'0.20.25'

A simple aggregation:

In [40]:
%%time
icustays_pl.group_by("subject_id").len().sort("subject_id")

CPU times: user 8.16 ms, sys: 7.71 ms, total: 15.9 ms
Wall time: 6.14 ms


subject_id,len
i64,u32
10000032,1
10000980,1
10001217,2
10001725,1
10001884,1
…,…
19999442,1
19999625,1
19999828,1


In [41]:
icustays_pl

subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
i64,i64,i64,str,str,str,str,f64
10000032,29079034,39553978,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2180-07-23 14:00:00""","""2180-07-23 23:50:47""",0.410266
10000980,26913865,39765666,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2189-06-27 08:42:00""","""2189-06-27 20:38:27""",0.497535
10001217,24597018,37067082,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2157-11-20 19:18:02""","""2157-11-21 22:08:00""",1.118032
10001217,27703517,34592300,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2157-12-19 15:42:24""","""2157-12-20 14:27:41""",0.948113
10001725,25563031,31205490,"""Medical/Surgical Intensive Car…","""Medical/Surgical Intensive Car…","""2110-04-11 15:52:22""","""2110-04-12 23:59:56""",1.338588
…,…,…,…,…,…,…,…
19999442,26785317,32336619,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2148-11-19 14:23:43""","""2148-11-26 13:12:15""",6.95037
19999625,25304202,31070865,"""Medical/Surgical Intensive Car…","""Medical/Surgical Intensive Car…","""2139-10-10 19:18:00""","""2139-10-11 18:21:28""",0.960741
19999828,25744818,36075953,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2149-01-08 18:12:00""","""2149-01-10 13:11:02""",1.790995


In [42]:
%%time
(icustays_pl.sort(["intime"]).group_by("subject_id")
 .first().sort("subject_id")
)

CPU times: user 56.4 ms, sys: 11.3 ms, total: 67.7 ms
Wall time: 23.2 ms


subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
i64,i64,i64,str,str,str,str,f64
10000032,29079034,39553978,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2180-07-23 14:00:00""","""2180-07-23 23:50:47""",0.410266
10000980,26913865,39765666,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2189-06-27 08:42:00""","""2189-06-27 20:38:27""",0.497535
10001217,24597018,37067082,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2157-11-20 19:18:02""","""2157-11-21 22:08:00""",1.118032
10001725,25563031,31205490,"""Medical/Surgical Intensive Car…","""Medical/Surgical Intensive Car…","""2110-04-11 15:52:22""","""2110-04-12 23:59:56""",1.338588
10001884,26184834,37510196,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2131-01-11 04:20:05""","""2131-01-20 08:27:30""",9.171817
…,…,…,…,…,…,…,…
19999442,26785317,32336619,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2148-11-19 14:23:43""","""2148-11-26 13:12:15""",6.95037
19999625,25304202,31070865,"""Medical/Surgical Intensive Car…","""Medical/Surgical Intensive Car…","""2139-10-10 19:18:00""","""2139-10-11 18:21:28""",0.960741
19999828,25744818,36075953,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2149-01-08 18:12:00""","""2149-01-10 13:11:02""",1.790995


In [43]:
%%time
icustays_pl.with_columns(
    pl.col("intime")
    .rank()
    .over("subject_id")
    .alias("rank")
).filter(pl.col("rank") == 1)

CPU times: user 111 ms, sys: 21.8 ms, total: 133 ms
Wall time: 54 ms


subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,rank
i64,i64,i64,str,str,str,str,f64,f64
10000032,29079034,39553978,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2180-07-23 14:00:00""","""2180-07-23 23:50:47""",0.410266,1.0
10000980,26913865,39765666,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2189-06-27 08:42:00""","""2189-06-27 20:38:27""",0.497535,1.0
10001217,24597018,37067082,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2157-11-20 19:18:02""","""2157-11-21 22:08:00""",1.118032,1.0
10001725,25563031,31205490,"""Medical/Surgical Intensive Car…","""Medical/Surgical Intensive Car…","""2110-04-11 15:52:22""","""2110-04-12 23:59:56""",1.338588,1.0
10001884,26184834,37510196,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2131-01-11 04:20:05""","""2131-01-20 08:27:30""",9.171817,1.0
…,…,…,…,…,…,…,…,…
19999442,26785317,32336619,"""Surgical Intensive Care Unit (…","""Surgical Intensive Care Unit (…","""2148-11-19 14:23:43""","""2148-11-26 13:12:15""",6.95037,1.0
19999625,25304202,31070865,"""Medical/Surgical Intensive Car…","""Medical/Surgical Intensive Car…","""2139-10-10 19:18:00""","""2139-10-11 18:21:28""",0.960741,1.0
19999828,25744818,36075953,"""Medical Intensive Care Unit (M…","""Medical Intensive Care Unit (M…","""2149-01-08 18:12:00""","""2149-01-10 13:11:02""",1.790995,1.0


## Web development (very quick intro)

Plotly company has a basic web development tool called [`Dash`](https://dash.plotly.com/). 

The below is the very basics of it:

In [44]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash()

app.layout = [
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.country.unique(), 'Canada', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='year', y='pop')

if __name__ == '__main__':
    app.run(debug=True)